### Визначення цільової функції та градієнта
Описуємо квадратичну функцію варіанта 9, її градієнт та готуємо сітку значень для візуалізацій.


In [6]:
import numpy as np


def f(x) -> np.ndarray:
    """Квадратична цільова функція варіанта 9."""
    x1, x2, x3 = x
    return (
        200 * x1 ** 2
        + 5 * x2 ** 2
        + 144 * x3 ** 2
        - 24 * x1 * x2
        - 48 * x1 * x3
        + 24 * x2 * x3
        + 5
    )


def grad_f(x) -> np.ndarray:
    """Аналітичний градієнт функції f."""
    x1, x2, x3 = x
    dfdx1 = 400 * x1 - 24 * x2 - 48 * x3
    dfdx2 = 10 * x2 - 24 * x1 + 24 * x3
    dfdx3 = 288 * x3 - 48 * x1 + 24 * x2
    return np.array([dfdx1, dfdx2, dfdx3], dtype=float)


grid = np.linspace(-1.2, 1.2, 12)
# Рівномірна сітка для подальших візуалізацій



### Інтерактивний зріз поверхні
Створюємо віджет, що дозволяє фіксувати одну змінну та досліджувати поверхню функції у двох вимірах.


In [ ]:
import plotly.graph_objects as go
import ipywidgets as w
from IPython.display import display


def surface_for_slice(slice_var="x3", slice_val=0.0):
    """Повертає поверхню f при фіксованому значенні однієї координати."""
    # Формуємо сітку для двох вільних змінних
    if slice_var == "x1":
        X2, X3 = np.meshgrid(grid, grid)
        Z = f((np.full_like(X2, slice_val), X2, X3))
        x, y, z = X2, X3, Z
        xlab, ylab = "x2", "x3"
    elif slice_var == "x2":
        X1, X3 = np.meshgrid(grid, grid)
        Z = f((X1, np.full_like(X1, slice_val), X3))
        x, y, z = X1, X3, Z
        xlab, ylab = "x1", "x3"
    else:  # slice_var == "x3"
        X1, X2 = np.meshgrid(grid, grid)
        Z = f((X1, X2, slice_val))
        x, y, z = X1, X2, Z
        xlab, ylab = "x1", "x2"

    fig = go.Figure(data=[go.Surface(x=x, y=y, z=z, showscale=False)])
    fig.update_layout(
        title=f"Surface of f with {slice_var} = {slice_val:.3f}",
        scene=dict(xaxis_title=xlab, yaxis_title=ylab, zaxis_title="f"),
        margin=dict(l=0, r=0, b=0, t=40),
        height=600,
    )
    return fig

# Віджети для вибору фіксованої координати та її значення
slice_var_dd = w.Dropdown(options=["x1", "x2", "x3"], value="x3", description="Fix:")
slice_val_sl = w.FloatSlider(
    value=0.0,
    min=grid.min(),
    max=grid.max(),
    step=float(grid[1] - grid[0]),
    description="Value:",
)

# Область, у яку рендеримо графік після кожної зміни параметрів
out = w.Output()


def _draw(*_):
    with out:
        out.clear_output(wait=True)
        display(surface_for_slice(slice_var_dd.value, slice_val_sl.value))

# Перемальовуємо поверхню при зміні будь-якого з параметрів
slice_var_dd.observe(_draw, names="value")
slice_val_sl.observe(_draw, names="value")

display(w.HBox([slice_var_dd, slice_val_sl]))
_draw()
display(out)



Output()

### Тривимірна сітка та ізоповерхня
Готуємо кубічну сітку, діапазони значень і віджет для побудови однієї ізоповерхні з виділенням мінімуму.


In [8]:
X1, X2, X3 = np.meshgrid(grid, grid, grid, indexing="xy")
F = f((X1, X2, X3))

# Межі значень для ползунка рівня ізоповерхні
Fmin, Fmax = np.percentile(F, 1), np.percentile(F, 10)

# Набір керуючих елементів для ізоповерхні
F_value = w.FloatSlider(
    value=float((Fmin + Fmax) / 2),
    min=float(Fmin),
    max=float(Fmax),
    step=float((Fmax - Fmin) / 200),
    description="F =",
    readout_format=".2f",
    continuous_update=True,
)
opacity = w.FloatSlider(value=0.6, min=0.1, max=1.0, step=0.05, description="Opacity")
showscale = w.Checkbox(value=True, description="Show colorbar")
btn = w.Button(description="Render", button_style="primary")
display(w.HBox([F_value, opacity, showscale, btn]))

out = w.Output()


def build_figure(F_level, opacity, showscale):
    """Формує одну ізоповерхню f(x) = F_level і позначає мінімум."""
    fig = go.Figure(
        go.Isosurface(
            x=X1.ravel(),
            y=X2.ravel(),
            z=X3.ravel(),
            value=F.ravel(),
            isomin=float(F_level),
            isomax=float(F_level),
            surface_count=1,
            opacity=float(opacity),
            caps=dict(x_show=False, y_show=False, z_show=False),
            colorscale="Viridis",
            showscale=bool(showscale),
            colorbar=dict(
                title="f",
                x=1.02,
                thickness=14,
                len=0.6,
                tickfont=dict(size=10),
            )
            if showscale
            else None,
        )
    )

    fig.update_layout(
        title=f"Isosurface: F = {F_level:.2f}",
        scene=dict(xaxis_title="x1", yaxis_title="x2", zaxis_title="x3"),
        margin=dict(l=0, r=80 if showscale else 0, b=0, t=40),
        height=650,
    )

    # Додаємо точку глобального мінімуму
    fig.add_scatter3d(
        x=[0],
        y=[0],
        z=[0],
        mode="markers+text",
        marker=dict(size=6, symbol="diamond"),
        text=["min f=5 at (0,0,0)"],
        textposition="top center",
        name="Global minimum",
    )

    # Додаємо кілька додаткових демонстраційних точок (якщо задані)
    # Створіть словник demo_points у будь-якій клітинці: {label: (x1,x2,x3), ...}
    pts = globals().get("demo_points", None)
    if isinstance(pts, dict) and len(pts) > 0:
        xs, ys, zs, texts = [], [], [], []
        for name, arr in pts.items():
            try:
                a = np.asarray(arr, dtype=float).reshape(-1)
                if a.size == 3:
                    xs.append(float(a[0]))
                    ys.append(float(a[1]))
                    zs.append(float(a[2]))
                    texts.append(str(name))
            except Exception:
                pass
        if xs:
            fig.add_scatter3d(
                x=xs, y=ys, z=zs,
                mode="markers+text",
                marker=dict(size=5, color="orange"),
                text=texts,
                textposition="top center",
                name="Demo starts",
            )

    return fig


def _render(_=None):
    with out:
        out.clear_output(wait=True)
        display(build_figure(F_value.value, opacity.value, showscale.value))

btn.on_click(_render)
_render()
display(out)



Output()

### Умова простого зменшення
Визначаємо критерій прийняття кроку, який вимагає лише зменшення значення функції.


In [9]:
def split_step_cond(f, x, p, t, f_x, c) -> bool:
    """Умова для методу з поділом кроку: достатньо, щоб f зменшилася."""
    return f(x + t * p) < f_x


### Умова Ґольдштейна
Впроваджуємо подвійні нерівності Ґольдштейна для контролю кроку лінійного пошуку.


In [10]:
def goldstein_cond(f, x, p, t, f_x, grad_f, c=0.1) -> bool:
    """
    Перевірка нерівностей Ґольдштейна:
      f(x) + (1-c) t pfi0 <= f(x + t p) <= f(x) + c t pfi0,  де pfi0 = df(x)^T p < 0.
    """
    g = grad_f(x)
    phi0 = float(np.dot(g, p))
    f_x_new = f(x + t * p)
    left = f_x + (1.0 - c) * t * phi0
    right = f_x + c * t * phi0
    return (left <= f_x_new) and (f_x_new <= right)

### Зворотний пошук кроку
Реалізуємо процедуру backtracking з підтримкою простого та ґольдштейнівського критеріїв.


In [11]:
from typing import Callable, Optional


def back_tracking(
    f: Callable[[np.ndarray], float],
    grad_f: Callable[[np.ndarray], np.ndarray],
    x: np.ndarray,
    p: np.ndarray,
    cond: Optional[Callable] = None,
    t0: float = 1.0,
    beta: float = 0.5,
    max_halves: int = 100,
    c: float = 0.1,
) -> float:
    """
    Пошук кроку:
      - якщо cond is goldstein_cond -> пошук за правилом Ґольдштейна
        (розширення до лівої межі, потім стискання до правої);
      - інакше -> "поділ кроку" (backtracking), поки cond(...) не стане True.
    beta ∈ (0,1) - коефіцієнт стискання; c є (0,0.5] - параметр Ґольдштейна.
    """
    t = float(t0)
    fx = float(f(x))
    g = grad_f(x)
    phi0 = float(np.dot(g, p))  # напрямна похідна у нулі

    # гарантуємо напрям спуску
    if phi0 >= 0:
        p = -g
        phi0 = -float(np.dot(g, g))
        if phi0 >= 0:  # нульовий градієнт
            return 0.0

    #  Goldstein 
    if cond is goldstein_cond:
        # 1) Розширення: забезпечити ліву нерівність
        for _ in range(max_halves):
            if f(x + t * p) >= fx + (1.0 - c) * t * phi0:
                break
            t /= beta  # beta<1 -> збільшуємо крок
        # 2) Стискання: забезпечити праву нерівність
        for _ in range(max_halves):
            if goldstein_cond(f, x, p, t, fx, grad_f, c=c):
                return t
            t *= beta
        return t

    # Поділ кроку (звичайний backtracking)
    # за замовчуванням приймаємо split_step_cond, якщо cond не задано
    if cond is None:
        cond = split_step_cond

    for _ in range(max_halves):
        if cond(f, x, p, t, fx, grad_f):
            return t
        t *= beta
    return t

### Евклідова норма
Додаємо коротку допоміжну функцію для обчислення норми градієнта.


In [12]:
def norm2(v: np.ndarray) -> float:
    """Евклідова норма вектора."""
    return np.sqrt(np.sum(v ** 2))



### Градієнтний спуск
Описуємо основний цикл методу зі збереженням історії проміжних оцінок.


In [13]:
def gd_back_tracking(
    f,
    grad_f,
    x0,
    cond,
    max_iters=50,
    tol=1e-4,
    c=0.5,
):
    """Градієнтний спуск із backtracking та збереженням історії."""
    x = np.array(x0, dtype=float)
    history = {"x": [x.copy()], "f": [f(x)], "t": [], "grad_norm": []}

    for _ in range(max_iters):
        g = grad_f(x)
        gnorm = norm2(g)
        history["grad_norm"].append(gnorm)
        if gnorm < tol:
            break

        p = -g
        assert np.dot(g, p) < 0, "Not a descent direction!"

        # Підбираємо довжину кроку відповідно до переданої умови
        t = back_tracking(f, grad_f, x, p, cond, c)
        assert t > 0, "Line search failed to find a valid step size!"

        x += t * p

        history["x"].append(x.copy())
        history["f"].append(f(x))
        history["t"].append(t)

    return x, history



### Перший запуск зі старту (1,1,1)
Запускаємо градієнтний спуск із простим правилом зменшення, щоб перевірити збіжність.


In [14]:
# Початкова точка для першого експерименту
x0 = np.array([1.0, 1.0, 1.0])

# Запуск градієнтного спуску з простою умовою зменшення
x_min, history = gd_back_tracking(f, grad_f, x0, split_step_cond, max_iters=500, tol=1e-4)
print(f"Found minimum at x = {x_min}, f = {f(x_min)}, iters = {len(history['f'])-1}")



Found minimum at x = [ 3.18700924e-07  9.21802411e-06 -6.45250994e-07], f = 5.000000000301742, iters = 319


### Адаптер виклику функції
Нормалізуємо інтерфейс цільової функції для модулів візуалізації.


In [29]:
import numpy as np


def f_adapter(f_raw):
    """Уніфікує виклики f для кортежів та списків координат."""
    def fA(x_tuple):
        try:
            return f_raw(x_tuple)
        except TypeError:
            x1, x2, x3 = x_tuple
            return f_raw(x1, x2, x3)

    return fA



### Ізоповерхня з траєкторією
Будуємо тривимірне представлення ізоповерхні та додаємо шлях оптимізації.


In [30]:
import plotly.graph_objects as go
import ipywidgets as w
from IPython.display import display


def show_isosurface_with_path(f_raw, history, grid_range=(-1.2, 1.2), vol_n=50):
    """Відображає ізоповерхню разом із траєкторією оптимізації."""
    f = f_adapter(f_raw)

    P = np.array(history["x"])  # (K+1, 3)

    gmin, gmax = grid_range
    vol_grid = np.linspace(gmin, gmax, int(vol_n))
    X1v, X2v, X3v = np.meshgrid(vol_grid, vol_grid, vol_grid, indexing="xy")
    Fvol = f((X1v, X2v, X3v))
    
    # Використовуємо вужчі межі: від мінімуму функції (5) до 30-го перцентиля
    Fmin = 5.0  # Відомий мінімум функції
    Fmax = np.percentile(Fvol, 30)  # Вужча верхня межа (замість 95)

    level = w.FloatSlider(
        value=float((Fmin + Fmax) / 2),
        min=float(Fmin),
        max=float(Fmax),
        step=float((Fmax - Fmin) / 200),
        description="F =",
        readout_format=".2f",
        continuous_update=True,
    )
    opacity = w.FloatSlider(
        value=0.45,
        min=0.1,
        max=1.0,
        step=0.05,
        description="Opacity",
        continuous_update=True,
    )
    cbar = w.Checkbox(value=True, description="Show colorbar")
    display(w.HBox([level, opacity, cbar]))

    figw = go.FigureWidget()
    figw.update_layout(
        title=f"Isosurface: F = {level.value:.2f}",
        height=650,
        margin=dict(l=0, r=80, b=0, t=40),
        scene=dict(xaxis_title="x1", yaxis_title="x2", zaxis_title="x3"),
    )

    iso = go.Isosurface(
        x=X1v.ravel(),
        y=X2v.ravel(),
        z=X3v.ravel(),
        value=Fvol.ravel(),
        isomin=float(level.value),
        isomax=float(level.value),
        surface_count=1,
        opacity=float(opacity.value),
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorscale="Viridis",
        showscale=bool(cbar.value),
        colorbar=dict(title="f", x=1.02, thickness=14, len=0.6, tickfont=dict(size=10)),
    )
    path = go.Scatter3d(
        x=P[:, 0],
        y=P[:, 1],
        z=P[:, 2],
        mode="lines+markers",
        name="Path",
        marker=dict(size=4),
        line=dict(width=4),
        hovertemplate="iter=%{customdata}<br>x1=%{x:.3f}, x2=%{y:.3f}, x3=%{z:.3f}<extra></extra>",
        customdata=np.arange(len(P)),
    )
    figw.add_traces([iso, path])
    display(figw)

    iso_tr = figw.data[0]

    def _on_level(ch):
        val = float(ch["new"])
        with figw.batch_update():
            iso_tr.isomin = val
            iso_tr.isomax = val
            figw.layout.title = f"Isosurface: F = {val:.2f}"

    def _on_opacity(ch):
        iso_tr.opacity = float(ch["new"])

    def _on_cbar(ch):
        iso_tr.showscale = bool(ch["new"])
        figw.layout.margin.r = 80 if iso_tr.showscale else 0

    level.observe(_on_level, names="value")
    opacity.observe(_on_opacity, names="value")
    cbar.observe(_on_cbar, names="value")

### Графіки збіжності
Генеруємо графіки зміни значення функції, кроків і норми градієнта.


In [31]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def plot_metrics(history, log_f=True, log_grad=True):
    """Відстежує f(x_k), кроки t_k та норму градієнта."""
    fvals = np.array(history.get("f", []))
    tvals = np.array(history.get("t", []))
    gnorm = np.array(history.get("grad_norm", []))

    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=("f(x_k)", "step size t_k", "||grad f(x_k)||"),
        specs=[[{"type": "scatter"}, {"type": "scatter"}], [{"type": "scatter"}, None]],
    )
    if fvals.size:
        fig.add_trace(go.Scatter(y=fvals, mode="lines+markers", name="f"), row=1, col=1)
        if log_f:
            fig.update_yaxes(type="log", row=1, col=1)
    if tvals.size:
        fig.add_trace(go.Scatter(y=tvals, mode="lines+markers", name="t"), row=1, col=2)
    if gnorm.size:
        fig.add_trace(go.Scatter(y=gnorm, mode="lines+markers", name="||grad||"), row=2, col=1)
        if log_grad:
            fig.update_yaxes(type="log", row=2, col=1)

    fig.update_layout(height=700, width=900, title_text="Optimization metrics")
    fig.update_xaxes(title_text="iteration", range=[0, 50], row=1, col=1)
    fig.update_xaxes(title_text="iteration", range=[0, 50], row=1, col=2)
    fig.update_xaxes(title_text="iteration", range=[0, 50], row=2, col=1)

    # fig.update_xaxes(title_text="iteration", range=[0, len(fvals)-1], row=1, col=1)
    # fig.update_xaxes(title_text="iteration", range=[0, len(tvals)-1], row=1, col=2)
    # fig.update_xaxes(title_text="iteration", range=[0, len(gnorm)-1], row=2, col=1)

    fig.update_yaxes(title_text="f", row=1, col=1)
    fig.update_yaxes(title_text="t", row=1, col=2)
    fig.update_yaxes(title_text="||grad f||", row=2, col=1)
    fig.show()

### Відео траєкторії
Формуємо анімацію шляху оптимізації та зберігаємо її у форматі MP4.


In [32]:
import matplotlib.pyplot as plt
from matplotlib import animation


def save_video_xyz(history, filename="path_xyz.mp4", fps=8):
    """Записує тривимірну траєкторію оптимізації у відео."""
    P = np.array(history["x"])
    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection="3d")
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_zlabel("x3")

    pad = 0.15
    ax.set_xlim(P[:, 0].min() - pad, P[:, 0].max() + pad)
    ax.set_ylim(P[:, 1].min() - pad, P[:, 1].max() + pad)
    ax.set_zlim(P[:, 2].min() - pad, P[:, 2].max() + pad)

    line, = ax.plot([], [], [], lw=2, c="tab:blue")
    point, = ax.plot([], [], [], "o", c="tab:red")

    def init():
        line.set_data([], [])
        line.set_3d_properties([])
        point.set_data([], [])
        point.set_3d_properties([])
        return line, point

    def update(i):
        line.set_data(P[: i + 1, 0], P[: i + 1, 1])
        line.set_3d_properties(P[: i + 1, 2])
        point.set_data(P[i, 0:1], P[i, 1:2])
        point.set_3d_properties(P[i, 2:3])
        return line, point

    ani = animation.FuncAnimation(
        fig,
        update,
        init_func=init,
        frames=len(P),
        interval=int(1000 / fps),
        blit=True,
    )
    ani.save(filename, writer="ffmpeg", fps=fps)
    plt.close(fig)
    print(f"Saved 3D trajectory video to {filename}")



### Візуалізації для простого правила
Виводимо тривимірні візуалізації та метрики після першого запуску.


In [19]:
show_isosurface_with_path(f, history, grid_range=(-1.2, 1.2), vol_n=50)
plot_metrics(history, log_f=True, log_grad=True)
save_video_xyz(history, filename="opt_path_simple.mp4", fps=8)



FigureWidget({
    'data': [{'caps': {'x': {'show': False}, 'y': {'show': False}, 'z': {'show': False}},
              'colorbar': {'len': 0.6, 'thickness': 14, 'tickfont': {'size': 10}, 'title': {'text': 'f'}, 'x': 1.02},
              'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
                             [0.2222222222222222, '#3e4989'], [0.3333333333333333,
                             '#31688e'], [0.4444444444444444, '#26828e'],
                             [0.5555555555555556, '#1f9e89'], [0.6666666666666666,
                             '#35b779'], [0.7777777777777778, '#6ece58'],
                             [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
              'isomax': 54.01307788421491,
              'isomin': 54.01307788421491,
              'opacity': 0.45,
              'showscale': True,
              'surface': {'count': 1},
              'type': 'isosurface',
              'uid': '2dddb272-11c5-4903-ad74-7526f108cba4',
              'v

Saved 3D trajectory video to opt_path_simple.mp4


### Другий запуск з умовою Ґольдштейна
Повторюємо оптимізацію з іншої стартової точки та іншим правилом пошуку кроку.


In [33]:
x0 = np.array([1, 1, 1.0])

# Запускаємо градієнтний спуск із умовою Ґольдштейна
x_min, history = gd_back_tracking(
    f,
    grad_f,
    x0,
    goldstein_cond,
    max_iters=500,
    tol=1e-4,
    c=0.7,
)
print(f"Found minimum at x = {x_min}, f = {f(x_min)}, iters = {len(history['f'])-1}")



Found minimum at x = [ 6.56451611e-07  1.13848467e-05 -8.97498024e-07], f = 5.000000000453936, iters = 160


### Візуалізації для Ґольдштейна
Порівнюємо результати другого запуску на ізоповерхні та графіках.


In [21]:
show_isosurface_with_path(f, history, grid_range=(-1.2, 1.2), vol_n=50)
plot_metrics(history, log_f=True, log_grad=True)
save_video_xyz(history, filename="opt_path_goldstein.mp4", fps=8)



FigureWidget({
    'data': [{'caps': {'x': {'show': False}, 'y': {'show': False}, 'z': {'show': False}},
              'colorbar': {'len': 0.6, 'thickness': 14, 'tickfont': {'size': 10}, 'title': {'text': 'f'}, 'x': 1.02},
              'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
                             [0.2222222222222222, '#3e4989'], [0.3333333333333333,
                             '#31688e'], [0.4444444444444444, '#26828e'],
                             [0.5555555555555556, '#1f9e89'], [0.6666666666666666,
                             '#35b779'], [0.7777777777777778, '#6ece58'],
                             [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
              'isomax': 54.01307788421491,
              'isomin': 54.01307788421491,
              'opacity': 0.45,
              'showscale': True,
              'surface': {'count': 1},
              'type': 'isosurface',
              'uid': '89d679f9-83c0-4860-8abc-50288a35a5e6',
              'v

Saved 3D trajectory video to opt_path_goldstein.mp4


### Додаткові стартові точки
Нижче визначені кілька репрезентативних точок для запуску алгоритмів. Вони також відображаються на інтерактивній ізоповерхні (натисніть Render, щоб оновити).


In [34]:
demo_points = {
    "A: середній масштаб (+ + +)": [1.0, 1.0, 1.0],
    "B: далекі точки (-10,10,1)": [-10.0, 10.0, 1.0],
    "C: змішані знаки (0.5,-0.8,0.2)": [0.5, -0.8, 0.2],
    "D: жорстка вісь x1 (3,0,0)": [3.0, 0.0, 0.0],
    "E: жорстка вісь x3 (0,0,3)": [0.0, 0.0, 3.0],
    "F: поблизу мінімуму (1e-3,-1e-3,2e-3)": [1e-3, -1e-3, 2e-3],
}
print("Додано", len(demo_points), "точок. Вони будуть показані на ізоповерхні.")


Додано 6 точок. Вони будуть показані на ізоповерхні.


### Порівняння методів на різних стартових точках
Запускаємо обидва методи (простий поділ кроку та Ґольдштейна) з різних початкових точок для порівняння.

In [23]:
import pandas as pd

# Точка A: середній масштаб (+ + +)
print("=" * 60)
print("Точка A: середній масштаб [1.0, 1.0, 1.0]")
print("=" * 60)

x0_A = np.array([1.0, 1.0, 1.0])

# Метод з простим поділом кроку
x_min_simple, hist_simple = gd_back_tracking(
    f, grad_f, x0_A, split_step_cond, max_iters=2000, tol=1e-4
)
print(f"\n[Простий поділ кроку]")
print(f"  Знайдено мінімум: x = {x_min_simple}")
print(f"  f(x) = {f(x_min_simple):.6f}")
print(f"  Кількість ітерацій: {len(hist_simple['t'])}")
print(f"  Середній крок: {np.mean(hist_simple['t']):.6f}")

# Метод Ґольдштейна
x_min_gold, hist_gold = gd_back_tracking(
    f, grad_f, x0_A, goldstein_cond, max_iters=2000, tol=1e-4, c=0.1
)
print(f"\n[Метод Ґольдштейна, c=0.1]")
print(f"  Знайдено мінімум: x = {x_min_gold}")
print(f"  f(x) = {f(x_min_gold):.6f}")
print(f"  Кількість ітерацій: {len(hist_gold['t'])}")
print(f"  Середній крок: {np.mean(hist_gold['t']):.6f}")

# Збереження результатів для підсумкової таблиці
results = []
results.append({
    "Точка": "A: (1,1,1)",
    "Метод": "Простий",
    "Ітерації": len(hist_simple['t']),
    "f(x*)": f(x_min_simple),
    "Середній крок": np.mean(hist_simple['t'])
})
results.append({
    "Точка": "A: (1,1,1)",
    "Метод": "Ґольдштейн",
    "Ітерації": len(hist_gold['t']),
    "f(x*)": f(x_min_gold),
    "Середній крок": np.mean(hist_gold['t'])
})

Точка A: середній масштаб [1.0, 1.0, 1.0]

[Простий поділ кроку]
  Знайдено мінімум: x = [ 3.18700924e-07  9.21802411e-06 -6.45250994e-07]
  f(x) = 5.000000
  Кількість ітерацій: 319
  Середній крок: 0.005131

[Метод Ґольдштейна, c=0.1]
  Знайдено мінімум: x = [ 4.63329104e-07  1.13267375e-05 -8.20302414e-07]
  f(x) = 5.000000
  Кількість ітерацій: 271
  Середній крок: 0.005893


In [24]:
# Точка B: далекі точки (-10, 10, 1)
print("\n" + "=" * 60)
print("Точка B: далекі точки [-10.0, 10.0, 1.0]")
print("=" * 60)

x0_B = np.array([-10.0, 10.0, 1.0])

# Метод з простим поділом кроку
x_min_simple, hist_simple = gd_back_tracking(
    f, grad_f, x0_B, split_step_cond, max_iters=2000, tol=1e-4
)
print(f"\n[Простий поділ кроку]")
print(f"  Знайдено мінімум: x = {x_min_simple}")
print(f"  f(x) = {f(x_min_simple):.6f}")
print(f"  Кількість ітерацій: {len(hist_simple['t'])}")
print(f"  Середній крок: {np.mean(hist_simple['t']):.6f}")

# Метод Ґольдштейна
x_min_gold, hist_gold = gd_back_tracking(
    f, grad_f, x0_B, goldstein_cond, max_iters=2000, tol=1e-4, c=0.1
)
print(f"\n[Метод Ґольдштейна, c=0.1]")
print(f"  Знайдено мінімум: x = {x_min_gold}")
print(f"  f(x) = {f(x_min_gold):.6f}")
print(f"  Кількість ітерацій: {len(hist_gold['t'])}")
print(f"  Середній крок: {np.mean(hist_gold['t']):.6f}")

results.append({
    "Точка": "B: (-10,10,1)",
    "Метод": "Простий",
    "Ітерації": len(hist_simple['t']),
    "f(x*)": f(x_min_simple),
    "Середній крок": np.mean(hist_simple['t'])
})
results.append({
    "Точка": "B: (-10,10,1)",
    "Метод": "Ґольдштейн",
    "Ітерації": len(hist_gold['t']),
    "f(x*)": f(x_min_gold),
    "Середній крок": np.mean(hist_gold['t'])
})


Точка B: далекі точки [-10.0, 10.0, 1.0]

[Простий поділ кроку]
  Знайдено мінімум: x = [ 3.44232302e-07  9.77838797e-06 -6.86831948e-07]
  f(x) = 5.000000
  Кількість ітерацій: 380
  Середній крок: 0.005130

[Метод Ґольдштейна, c=0.1]
  Знайдено мінімум: x = [ 6.94183033e-07  1.10409301e-05 -8.92411659e-07]
  f(x) = 5.000000
  Кількість ітерацій: 325
  Середній крок: 0.005904


In [25]:
# Точка C: змішані знаки (0.5, -0.8, 0.2)
print("\n" + "=" * 60)
print("Точка C: змішані знаки [0.5, -0.8, 0.2]")
print("=" * 60)

x0_C = np.array([0.5, -0.8, 0.2])

x_min_simple, hist_simple = gd_back_tracking(
    f, grad_f, x0_C, split_step_cond, max_iters=2000, tol=1e-4
)
print(f"\n[Простий поділ кроку]")
print(f"  Ітерації: {len(hist_simple['t'])}, f(x*) = {f(x_min_simple):.6f}, середній крок: {np.mean(hist_simple['t']):.6f}")

x_min_gold, hist_gold = gd_back_tracking(
    f, grad_f, x0_C, goldstein_cond, max_iters=2000, tol=1e-4, c=0.1
)
print(f"[Метод Ґольдштейна]")
print(f"  Ітерації: {len(hist_gold['t'])}, f(x*) = {f(x_min_gold):.6f}, середній крок: {np.mean(hist_gold['t']):.6f}")

results.extend([
    {"Точка": "C: (0.5,-0.8,0.2)", "Метод": "Простий", "Ітерації": len(hist_simple['t']), "f(x*)": f(x_min_simple), "Середній крок": np.mean(hist_simple['t'])},
    {"Точка": "C: (0.5,-0.8,0.2)", "Метод": "Ґольдштейн", "Ітерації": len(hist_gold['t']), "f(x*)": f(x_min_gold), "Середній крок": np.mean(hist_gold['t'])}
])

# Точка D: жорстка вісь x1 (3, 0, 0)
print("\n" + "=" * 60)
print("Точка D: жорстка вісь x1 [3.0, 0.0, 0.0]")
print("=" * 60)

x0_D = np.array([3.0, 0.0, 0.0])

x_min_simple, hist_simple = gd_back_tracking(
    f, grad_f, x0_D, split_step_cond, max_iters=2000, tol=1e-4
)
print(f"\n[Простий поділ кроку]")
print(f"  Ітерації: {len(hist_simple['t'])}, f(x*) = {f(x_min_simple):.6f}, середній крок: {np.mean(hist_simple['t']):.6f}")

x_min_gold, hist_gold = gd_back_tracking(
    f, grad_f, x0_D, goldstein_cond, max_iters=2000, tol=1e-4, c=0.1
)
print(f"[Метод Ґольдштейна]")
print(f"  Ітерації: {len(hist_gold['t'])}, f(x*) = {f(x_min_gold):.6f}, середній крок: {np.mean(hist_gold['t']):.6f}")

results.extend([
    {"Точка": "D: (3,0,0)", "Метод": "Простий", "Ітерації": len(hist_simple['t']), "f(x*)": f(x_min_simple), "Середній крок": np.mean(hist_simple['t'])},
    {"Точка": "D: (3,0,0)", "Метод": "Ґольдштейн", "Ітерації": len(hist_gold['t']), "f(x*)": f(x_min_gold), "Середній крок": np.mean(hist_gold['t'])}
])


Точка C: змішані знаки [0.5, -0.8, 0.2]

[Простий поділ кроку]
  Ітерації: 313, f(x*) = 5.000000, середній крок: 0.005129
[Метод Ґольдштейна]
  Ітерації: 264, f(x*) = 5.000000, середній крок: 0.005907

Точка D: жорстка вісь x1 [3.0, 0.0, 0.0]

[Простий поділ кроку]
  Ітерації: 266, f(x*) = 5.000000, середній крок: 0.005081
[Метод Ґольдштейна]
  Ітерації: 228, f(x*) = 5.000000, середній крок: 0.005866


In [26]:
# Точка E: жорстка вісь x3 (0, 0, 3)
print("\n" + "=" * 60)
print("Точка E: жорстка вісь x3 [0.0, 0.0, 3.0]")
print("=" * 60)

x0_E = np.array([0.0, 0.0, 3.0])

x_min_simple, hist_simple = gd_back_tracking(
    f, grad_f, x0_E, split_step_cond, max_iters=2000, tol=1e-4
)
print(f"\n[Простий поділ кроку]")
print(f"  Ітерації: {len(hist_simple['t'])}, f(x*) = {f(x_min_simple):.6f}, середній крок: {np.mean(hist_simple['t']):.6f}")

x_min_gold, hist_gold = gd_back_tracking(
    f, grad_f, x0_E, goldstein_cond, max_iters=2000, tol=1e-4, c=0.1
)
print(f"[Метод Ґольдштейна]")
print(f"  Ітерації: {len(hist_gold['t'])}, f(x*) = {f(x_min_gold):.6f}, середній крок: {np.mean(hist_gold['t']):.6f}")

results.extend([
    {"Точка": "E: (0,0,3)", "Метод": "Простий", "Ітерації": len(hist_simple['t']), "f(x*)": f(x_min_simple), "Середній крок": np.mean(hist_simple['t'])},
    {"Точка": "E: (0,0,3)", "Метод": "Ґольдштейн", "Ітерації": len(hist_gold['t']), "f(x*)": f(x_min_gold), "Середній крок": np.mean(hist_gold['t'])}
])

# Точка F: поблизу мінімуму (1e-3, -1e-3, 2e-3)
print("\n" + "=" * 60)
print("Точка F: поблизу мінімуму [1e-3, -1e-3, 2e-3]")
print("=" * 60)

x0_F = np.array([1e-3, -1e-3, 2e-3])

x_min_simple, hist_simple = gd_back_tracking(
    f, grad_f, x0_F, split_step_cond, max_iters=2000, tol=1e-4
)
print(f"\n[Простий поділ кроку]")
print(f"  Ітерації: {len(hist_simple['t'])}, f(x*) = {f(x_min_simple):.6f}, середній крок: {np.mean(hist_simple['t']):.6f}")

x_min_gold, hist_gold = gd_back_tracking(
    f, grad_f, x0_F, goldstein_cond, max_iters=2000, tol=1e-4, c=0.1
)
print(f"[Метод Ґольдштейна]")
print(f"  Ітерації: {len(hist_gold['t'])}, f(x*) = {f(x_min_gold):.6f}, середній крок: {np.mean(hist_gold['t']):.6f}")

results.extend([
    {"Точка": "F: (~0,~0,~0)", "Метод": "Простий", "Ітерації": len(hist_simple['t']), "f(x*)": f(x_min_simple), "Середній крок": np.mean(hist_simple['t'])},
    {"Точка": "F: (~0,~0,~0)", "Метод": "Ґольдштейн", "Ітерації": len(hist_gold['t']), "f(x*)": f(x_min_gold), "Середній крок": np.mean(hist_gold['t'])}
])


Точка E: жорстка вісь x3 [0.0, 0.0, 3.0]

[Простий поділ кроку]
  Ітерації: 278, f(x*) = 5.000000, середній крок: 0.005101
[Метод Ґольдштейна]
  Ітерації: 235, f(x*) = 5.000000, середній крок: 0.005878

Точка F: поблизу мінімуму [1e-3, -1e-3, 2e-3]

[Простий поділ кроку]
  Ітерації: 131, f(x*) = 5.000000, середній крок: 0.005099
[Метод Ґольдштейна]
  Ітерації: 110, f(x*) = 5.000000, середній крок: 0.005881


### Зведена таблиця результатів
Порівняння ефективності обох методів на всіх тестових точках.

In [27]:
# Створення та відображення підсумкової таблиці
df_results = pd.DataFrame(results)
df_results = df_results.round({"f(x*)": 6, "Середній крок": 6})

print("\n" + "=" * 80)
print("ПІДСУМКОВА ТАБЛИЦЯ ПОРІВНЯННЯ МЕТОДІВ")
print("=" * 80)
print(df_results.to_string(index=False))
print("=" * 80)

# Групування за методом для аналізу
print("\n" + "Середні показники по методах:")
print("-" * 80)
grouped = df_results.groupby("Метод").agg({
    "Ітерації": ["mean", "std"],
    "Середній крок": ["mean", "std"]
})
print(grouped.round(3))

# Виділення кращих результатів для кожної точки
print("\n" + "Порівняння по точках (менше ітерацій = краще):")
print("-" * 80)
for point in df_results["Точка"].unique():
    subset = df_results[df_results["Точка"] == point]
    best_method = subset.loc[subset["Ітерації"].idxmin(), "Метод"]
    iters_simple = subset[subset["Метод"] == "Простий"]["Ітерації"].values[0]
    iters_gold = subset[subset["Метод"] == "Ґольдштейн"]["Ітерації"].values[0]
    diff = abs(iters_simple - iters_gold)
    print(f"  {point}: {best_method} (різниця: {diff} ітерацій)")


ПІДСУМКОВА ТАБЛИЦЯ ПОРІВНЯННЯ МЕТОДІВ
            Точка      Метод  Ітерації  f(x*)  Середній крок
       A: (1,1,1)    Простий       319    5.0       0.005131
       A: (1,1,1) Ґольдштейн       271    5.0       0.005893
    B: (-10,10,1)    Простий       380    5.0       0.005130
    B: (-10,10,1) Ґольдштейн       325    5.0       0.005904
C: (0.5,-0.8,0.2)    Простий       313    5.0       0.005129
C: (0.5,-0.8,0.2) Ґольдштейн       264    5.0       0.005907
       D: (3,0,0)    Простий       266    5.0       0.005081
       D: (3,0,0) Ґольдштейн       228    5.0       0.005866
       E: (0,0,3)    Простий       278    5.0       0.005101
       E: (0,0,3) Ґольдштейн       235    5.0       0.005878
    F: (~0,~0,~0)    Простий       131    5.0       0.005099
    F: (~0,~0,~0) Ґольдштейн       110    5.0       0.005881

Середні показники по методах:
--------------------------------------------------------------------------------
           Ітерації         Середній крок     
         